In [2]:
%run -m ipy_startup

In [15]:
x = np.array([1,2,3,4])

In [17]:
x[:-2]

array([1, 2])

In [21]:
np.concatenate((np.array([1,23]), np.array([1,34])))

array([ 1, 23,  1, 34])

In [173]:
np.diff(np.array([1,2,3]))

array([1, 1])

In [349]:
from scipy import optimize

def sigmoid(v):
    r = 1 / (1 + np.exp(-v))
    if np.any(~np.isfinite(r)):
        print('Not finite for v: ', v)
    return r

class Regressor(object):
    
    def __init__(self, k):
        self.k = k
        
    def _objective(self, pv, X, y):
        
        # Parameters given as linear params first, then K-1 intercept terms
        pvb = pv[:-(self.k-1)]
        pva = pv[-(self.k-1):]
        
        if np.any(np.diff(pva) <= 0):
            print('found unordered pva: ', pva)
            return 1e10
        
        lp = 0
        for i in range(len(y)):
            k = y[i]
            Xi = X[i]
            
            s0 = 1 if k == self.k else sigmoid(np.dot(Xi, pvb) + pva[k-1])
            s1 = 0 if k == 1 else sigmoid(np.dot(Xi, pvb) + pva[k-2])
            assert 0 <= s0 <= 1
            assert 0 <= s1 <= 1
            p = s0 - s1
            assert p >= 0
            if p == 0:
                print('p 0')
            lp += np.log(p)
            
        # print('params: ', pv)
        # print('logprob: ', -lp)
        return -lp

    def _jacobian(self, pv, X, y):
        # Assume y is 1 through K
        
        assert np.all(np.in1d(y, np.arange(1, self.k + 1)))
        
        # Parameters given as linear params first, then K-1 intercept terms
        pvb = pv[:-(self.k-1)]
        pva = pv[-(self.k-1):]
        
        # Initialize gradient vectors to accmulate in loop
        gb = np.zeros(X.shape[1])
        ga = np.zeros(self.k - 1)
        
        for i in range(len(y)):
            k = y[i]
            Xi = X[i]
            
            # Helper values for gradient calculation
            s0 = 1 if k == self.k else sigmoid(np.dot(Xi, pvb) + pva[k-1])
            s1 = 0 if k == 1 else sigmoid(np.dot(Xi, pvb) + pva[k-2])
            spq0 = s0 * ( 1 - s0 )
            spq1 = s1 * ( 1 - s1 )
            
            # Accumulate gradient components
            if k < self.k:
                ga[k-1] += spq0 / ( s0 - s1 )
            if k > 1:
                ga[k-2] -= spq1 / ( s0 - s1 )
            gb += (Xi * (spq0 - spq1)) / (s0 - s1)
        
        
        j = np.concatenate((gb, ga))
        print('jac', j)
        return -j
    
    def _constraint(self, i):
        def con_val(pv):
            return pv[i] - pv[i-1]

        def con_jac(pv):
            j = np.zeros(len(pv))
            j[i] = 1.0
            j[i-1] = -1.0
            return j

        return {'type': 'ineq', 'fun': con_val, 'jac': con_jac}

#     def check_gradient(self, X, y):
#         err = []
        
#         e = optimize.check_grad(
#             self._objective,
#             self._jacobian,
#             pv,
#             X, y
#         )
#         self.err.append(err)
        
    def fit(self, X, y):
        pa0 = np.linspace(-1, 1, num=self.k-1)
        pb0 = np.zeros(X.shape[1])
        pv0 = np.concatenate((pb0, pa0))
        
        constraints = [self._constraint(i) for i in range(X.shape[1]+1, len(pv0))]
        
        self.err_ = []
        def callback(pv):
            print('callback')
            err = optimize.check_grad(
                self._objective,
                self._jacobian,
                pv,
                X, y
            )
            print('pv', pv)
            self.err_.append((err, np.array(pv)))
            
        self.optimize_result_ = optimize.minimize(
            self._objective,
            pv0,
            jac=self._jacobian,
            args=(X, y),
            #bounds=bounds,
            method='SLSQP',
            constraints=constraints,
            callback=callback
        )
        
        self.err_ = pd.DataFrame(self.err_, columns=['error', 'parameters'])
        
    def predict(self, X):
        n = len(X)
        pv = self.optimize_result_.x
        pvb = pv[:-(self.k-1)]
        pva = pv[-(self.k-1):]
        yb = np.dot(X, pvb)[:,np.newaxis]
        ya = np.tile(pva, (n, 1))
        p = sigmoid(ya + yb)
        assert p.ndim > 1
        p = np.hstack(( np.zeros((n, 1)), p, np.ones((n, 1)) ))
        return np.diff(p, axis=1)

In [350]:
from sklearn.datasets import load_iris
d = load_iris()
X, y = pd.DataFrame(d['data'], columns=d['feature_names']), pd.Series(d['target'] + 1)

In [351]:
y.value_counts()

3    50
2    50
1    50
dtype: int64

In [352]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [353]:
est = Regressor(3)
est.fit(X.values, y.values)

jac [ -57.82673357   16.22950045 -149.42837347  -65.13731936   15.27997573
  -15.27997573]
callback
jac [ nan  nan  nan  nan  nan  nan]
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0


/Users/eczech/anaconda/envs/research3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/Users/eczech/anaconda/envs/research3.5/lib/python3.5/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in double_scalars
/Users/eczech/anaconda/envs/research3.5/lib/python3.5/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in double_scalars
/Users/eczech/anaconda/envs/research3.5/lib/python3.5/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in true_divide
/Users/eczech/anaconda/envs/research3.5/lib/python3.5/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log
/Users/eczech/anaconda/envs/research3.5/lib/python3.5/site-packages/scipy/optimize/optimize.py:562: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/ecze

p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
p 0
jac [ 25.4782068   10.69214711  23.01763428   6.7425779  -21.72357336
  25.90174764]
callback
jac [ -1.48763258e+02  -6.91163937e+01  -1.21759793e+02  -4.47005852e+01
   1.81253898e-02  -2.31663621e+01]
pv [  5.02534043   9.30835672 -12.49356226  -7.96583565 -21.36619764
  24.00190947]
jac [-24.59057917 -12.00481758 -19.95335647  -8.12012629  -0.06280022
  -3.61248535]
callback
jac [-15.440892    -7.33950482 -12.46819187  -5.03788143  -0.07153095
  -2.14721297]
pv [ 3.05556474  3.89457068 -5.94040353 -4.80333537 -5.88891306  7.38797201]
jac [-15.440892    -7.33950482 -12.46819187  -5.03788143  -0.07153095
  -2.14721297]
callback
jac [-6.05410867 -2.53563779 -4.90804778 -1.95991938 -0.08349004 -0.62510272]
pv [ 3.50142966  3.13937092 -5.60135504 -6.46368552 -5.58086874  7.62553428]
jac [-6.05410867 -2.53563779 -4.90804778 -1.95991938 -0.08349004 -0.62510272]
callback
jac [-2.96767821 -0.99958413 -2.46614694 -0.9711283  -0.08160

jac [  4.70236863e-01   2.27777914e-01   3.99291055e-01   1.33959189e-01
  -1.34182820e-05   8.48517277e-02]
pv [  2.66911424   6.41846497  -9.24504426 -17.43217658   5.51636621
  39.74059433]
jac [  4.70236863e-01   2.27777914e-01   3.99291055e-01   1.33959189e-01
  -1.34182820e-05   8.48517277e-02]
callback
jac [  2.59409881e-01   1.20682211e-01   2.09033937e-01   7.11757701e-02
  -1.21983753e-05   4.60646720e-02]
pv [  2.50723513   6.49060498  -9.13091179 -17.63362418   5.9474521
  40.32951602]
jac [  2.59409881e-01   1.20682211e-01   2.09033937e-01   7.11757701e-02
  -1.21983753e-05   4.60646720e-02]
callback
jac [  1.65362692e-02   2.92139290e-03   7.24433954e-03   2.38985213e-03
  -9.10913401e-06   3.45205417e-03]
pv [  2.43772526   6.61851764  -9.22597097 -17.99265166   6.38881073
  41.48810937]
jac [  1.65362692e-02   2.92139290e-03   7.24433954e-03   2.38985213e-03
  -9.10913401e-06   3.45205417e-03]
callback
jac [ -5.63394125e-02  -2.94326974e-02  -4.79692095e-02  -1.67034074

In [354]:
est.optimize_result_

     fun: 5.9492814323239065
     jac: array([  6.15962335e-03,   2.93501175e-03,   4.83942576e-03,
         1.69139198e-03,   6.88615028e-06,   9.88924373e-04,
         0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 46
     nit: 37
    njev: 37
  status: 0
 success: True
       x: array([  2.46562861,   6.68282197,  -9.43111804, -18.28911399,
         6.76940527,  42.64389706])

In [355]:
pd.set_option('display.max_colwidth', 1000)
est.err_

,error,parameters
0,NaN,"[-57.8267335696, 16.2295004456, -149.428373472, -65.1373193559, -1.36068933898e-12, 1.33049127271e-12]"
1,NaN,"[67.7058388819, 111.514106289, -143.156179567, -75.5848026204, 14.0194942518, 14.0194942518]"
2,4.696126e-05,"[5.02534042621, 9.30835671942, -12.4935622636, -7.96583564646, -21.3661976407, 24.0019094729]"
3,2.172023e-06,"[3.05556474003, 3.89457068188, -5.94040352827, -4.80333536608, -5.88891305938, 7.3879720051]"
4,1.855478e-06,"[3.50142966294, 3.13937091935, -5.60135504412, -6.46368551932, -5.58086873707, 7.62553428311]"
5,1.294224e-06,"[3.5434331524, 3.02387969611, -5.39367541548, -7.25937295169, -5.39938645747, 7.89505470636]"
6,1.115026e-06,"[3.41632296108, 3.14693506758, -5.10211029667, -8.24538758727, -5.13427028294, 8.43487172486]"
7,1.033464e-06,"[3.08660940447, 3.57168016687, -4.74229806749, -9.42767014906, -4.75994147826, 9.37105628881]"
8,1.390549e-06,"[2.6625273179, 4.17929717348, -4.45464766314, -10.4991846073, -4.35010274596, 10.5880376434]"
9,1.657813e-06,"[2.30696090473, 4.75697155537, -4.35088028175, -11.2558161051, -3.97670235082, 11.9070653886]"


In [356]:
est.err_[est.err_['error'] > 1]

,error,parameters


In [358]:
y_pred = est.predict(X)
y_pred[:5]

array([[  1.00000000e+00,   5.80335779e-12,   0.00000000e+00],
       [  1.00000000e+00,   2.68528755e-10,   0.00000000e+00],
       [  1.00000000e+00,   4.49884574e-11,   0.00000000e+00],
       [  9.99999999e-01,   7.40611350e-10,   0.00000000e+00],
       [  1.00000000e+00,   3.80651066e-12,   0.00000000e+00]])

In [362]:
pd.DataFrame({'yp': np.argmax(y_pred, axis=1), 'yt': y}).groupby(['yp', 'yt']).size().unstack()

yt,1,2,3
yp,,,
0,50.0,NaN,NaN
1,NaN,49.0,1.0
2,NaN,1.0,49.0


In [193]:
y.value_counts()

3    50
2    50
1    50
dtype: int64